In [1]:
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv('restringido_interrumpido.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
0,203,SI,VIGENTE,TRÁNSITO RESTRINGIDO,08/01/2023,17/01/2023 08:00 HORAS,MADRE DE DIOS/TAMBOPATA/TAMBOPATA,KM 402 Y KM 409,(SECTOR SAN BERNARDO) PARO INDEFINIDO SOLICI...,-12.6898325,...,-12.6898325+-69.4042951,2023 08:00 HORAS+01+17+08:00,0,0,CEOPOL,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
2,128,SI,VIGENTE,TRÁNSITO RESTRINGIDO,06/01/2023,17/01/2023 08:00 HORAS,CUSCO/LA CONVENCION/MARANURA,KM 215,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.967832687702,...,-12.967832687702+-72.659684089566,2023 08:00 HORAS+01+17+08:00,0,0,DESPRCAR QUILLABAMBA,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,restringido
3,123,SI,VIGENTE,TRÁNSITO RESTRINGIDO,05/01/2023,17/01/2023 08:00 HORAS,CUSCO/LA CONVENCION/HUAYOPATA,KM 195,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.999269918989,...,-12.999269918989+-72.546403533388,2023 08:00 HORAS+01+17+08:00,0,1,DESPRCAR QUILLABAMBA,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,restringido
4,124,SI,VIGENTE,TRÁNSITO RESTRINGIDO,05/01/2023,17/01/2023 08:00 HORAS,CUSCO/LA CONVENCION/HUAYOPATA,KM 200,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-13.009372213976,...,-13.009372213976+-72.580577687622,2023 08:00 HORAS+01+17+08:00,0,1,DESPRCAR QUILLABAMBA,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,restringido
5,199,SI,VIGENTE,TRÁNSITO RESTRINGIDO,13/01/2023,17/01/2023 08:00 HORAS,MADRE DE DIOS/TAMBOPATA/INAMBARI,KM 265,(PUENTE PUENTE DOS DE MAYO PARO INDEFINIDO SO...,-13.065658,...,-13.065658+-70.3524946,2023 08:00 HORAS+01+17+08:00,0,0,CEOPOL,HUMANO,PE-30C,CARRETERA URCOS - MAZUCO - IÑAPARI,SI,restringido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,268,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,11/01/2023,17/01/2023 08:00 HORAS,SAN MARTIN/MOYOBAMBA/CALZADA,KM 483,(SECTOR CRUCE DE CALZADA) PARO INDEFINIDO SOL...,-6.039916695031,...,-6.039916695031+-77.060187746989,2023 08:00 HORAS+01+17+08:00,0,0,UD SAN MARTIN,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
117,265,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,12/01/2023,17/01/2023 08:00 HORAS,SAN MARTIN/LAMAS/ALONSO ALVARADO,KM 535,(SECTOR SAN JUAN DE PACAYZAPA) PARO INDEFINID...,-6.279584968,...,-6.279584968+-76.76164157,2023 08:00 HORAS+01+17+08:00,0,0,UD SAN MARTIN,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
118,183,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,16/01/2023,17/01/2023 08:00 HORAS,LA LIBERTAD/SANCHEZ CARRION/HUAMACHUCO,1108,(SECTOR LAGUNA SAUSACOCHA) PARO INDEFINIDO SOL...,-7.799640959,...,-7.799640959+-77.98079521,2023 08:00 HORAS+01+17+08:00,0,0,CONCESIONARIA CONVIAL SIERRA NORTE,HUMANO,PE-3N,LONGITUDINAL DE LA SIERRA NORTE,SI,interrumpido
119,184,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,16/01/2023,17/01/2023 08:00 HORAS,LA LIBERTAD/SANCHEZ CARRION/HUAMACHUCO,1100,(SECTOR LA CUCHILLA) PARO INDEFINIDO SOLICITAN...,-7.821478781,...,-7.821478781+-78.02545525,2023 08:00 HORAS+01+17+08:00,0,1,CONCESIONARIA CONVIAL SIERRA NORTE,HUMANO,PE-3N,LONGITUDINAL DE LA SIERRA NORTE,SI,interrumpido
